In [1]:
import pandas as pd

file_path = '03 Python test and Dataset.xlsx'
pricing_df = pd.read_excel(file_path, sheet_name='pricing_project_dataset')
platform_df = pd.read_excel(file_path, sheet_name='platform_number')
platform_df.columns = ['grass_region', 'platform_order', 'platform_gmv_usd']

Question 1

In [2]:
pricing_df['price_diff'] = pricing_df['competitor_model_price'] - pricing_df['shopee_model_price']
Q1 = pricing_df['price_diff'].quantile(0.25)
Q3 = pricing_df['price_diff'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers_df = pricing_df[
    (pricing_df['price_diff'] < lower_bound) | 
    (pricing_df['price_diff'] > upper_bound)
]

outlier_counts = outliers_df.groupby('grass_region').size().reset_index(name='outlier_count')
outlier_counts = outlier_counts.sort_values('outlier_count', ascending=False)
outlier_counts

,grass_region,outlier_count
0,ID,7595
1,VN,1952


In [3]:
regional_summary = pricing_df.groupby('grass_region').agg(
    total_shopee_order=('shopee_order', 'sum'),
    avg_price_diff=('price_diff', 'mean'),
    num_items=('shopee_item_id', 'nunique')
).reset_index()

In [4]:
merged = pd.merge(regional_summary, platform_df, on='grass_region', how='left')
merged['order_coverage_by_item'] = merged['total_shopee_order'] / merged['platform_order']
merged['net_competitiveness_by_item'] = merged['avg_price_diff']

final_result = merged[['grass_region', 'order_coverage_by_item', 'net_competitiveness_by_item', 'num_items']]
final_result

,grass_region,order_coverage_by_item,net_competitiveness_by_item,num_items
0,ID,0.546299,2565.841599,8428
1,MY,0.623496,0.008971,2118
2,PH,0.516156,-3.081481,2025
3,SG,0.417392,-0.738833,2194
4,TH,0.400668,5.296833,4137
5,VN,0.516903,-10502.506915,2097


Question 2

In [5]:
df = pricing_df

In [6]:
priority_map = {
    'Shopee < CPT': 0,
    'Shopee = CPT': 1,
    'Shopee > CPT': 2
}

df['priority'] = df['shopee_model_competitiveness_status'].map(priority_map).fillna(3)
df_sorted = df.sort_values(by=['priority', 'shopee_item_id'])
item_priority = df_sorted.drop_duplicates(subset='shopee_item_id', keep='first')

result = item_priority[['shopee_item_id', 'shopee_model_competitiveness_status']]
result

,shopee_item_id,shopee_model_competitiveness_status
8853,1323167,Shopee < CPT
10089,1837669,Shopee < CPT
4807,2650112,Shopee < CPT
8816,2666411,Shopee < CPT
19134,2794532,Shopee < CPT
...,...,...
16185,9997519744,Shopee > CPT
18895,9997666673,Shopee > CPT
4476,9998345075,Shopee > CPT
17944,9998445880,Shopee > CPT


Question 3

In [7]:
def top_30_percent_items_count(region_df):
    top_n = int(len(region_df) * 0.3)
    top_n = max(top_n, 1)
    top_models = region_df.head(top_n)
    return len(top_models['shopee_item_id'])

top_items_by_region = pricing_df.groupby('grass_region').apply(top_30_percent_items_count).reset_index()
top_items_by_region.columns = ['grass_region', 'total_items_in_top_30_percent_models']
top_items_by_region = top_items_by_region.sort_values('total_items_in_top_30_percent_models', ascending=False)

top_items_by_region

,grass_region,total_items_in_top_30_percent_models
0,ID,2528
4,TH,1241
3,SG,658
1,MY,635
5,VN,629
2,PH,607
